# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI" 

Embark on a journey to mastery with DeepLearning.AI, a beacon of innovation in the world of artificial intelligence. Offering cutting-edge courses, including neural networks and deep learning specialization, DeepLearning.AI equips learners with the tools to revolutionize industries and drive technological advancements. With a focus on hands-on learning and expert instruction from industry leaders like Andrew Ng, this platform empowers students to delve into the depths of deep learning. Whether you're a seasoned professional or a curious beginner, DeepLearning.AI opens doors to a future where AI transforms the world. Dive in and unlock your potential today!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI" 

Embark on a journey to mastery with DeepLearning.AI, a beacon of innovation in the world of artificial intelligence. Offering cutting-edge courses, including neural networks and deep learning specialization, DeepLearning.AI equips learners with the tools to revolutionize industries and drive technological advancements. With a focus on hands-on learning and expert instruction from industry leaders like Andrew Ng, this platform empowers students to delve into the depths of deep learning. Whether you're a seasoned professional or a curious beginner, DeepLearning.AI opens doors to a future where AI transforms the world. Dive in and unlock your potential today!

----

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI" 

Embark on a journey to mastery with DeepLearning.AI, a beacon of innovation in the world of artificial intelligence. Offering cutting-edge courses, including neural networks and deep learning specialization, DeepLearning.AI equips learners with the tools to revolutionize industries and drive technological advancements. With a focus on hands-on learning and expert instruction from industry leaders like Andrew Ng, this platform empowers students to delve into the depths of deep learning. Whether you're a seasoned professional or a curious beginner, DeepLearning.AI opens doors to a future where AI transforms the world. Dive in and unlock your potential today!

----


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Based on the aggregated feedback from the reviewers, it is evident that the content effectively highlights the key features and benefits of DeepLearning.AI, encouraging readers to explore the platform. The tone is engaging, persuasive, and motivates individuals to take action and enroll in the courses offered. Overall, the content is well-written and compelling, as noted by all the reviewers.

Final Suggestion:
The writing has successfully conveyed the message and purpose of promoting DeepLearning.AI. To further enhance the content, it could be beneficial to add some specific examples or success stories of individuals who have benefited from the courses. This would provide a more personalized touch and strengthen the persuasive aspect of the content. Overall, the piece is strong and effectively achieves its goal of engaging readers to consider enrolling in DeepLearning.AI cours

## Get the summary

In [15]:
print(res.summary)

Title: "Elevate Your Skills: The Deep Learning Revolution at DeepLearning.AI"

Unlock the world of artificial intelligence and propel your career with DeepLearning.AI. Led by industry expert Andrew Ng, this platform offers transformative courses in deep learning and neural networks. Dive into hands-on learning experiences that empower you to shape the future of technology. Join a community of learners pushing boundaries and driving innovation. Whether you're a novice or a seasoned pro, DeepLearning.AI paves the way for your success. Don't just dream of revolutionizing industries—make it a reality with DeepLearning.AI. Enroll today and unleash your potential!
